In [ ]:
# ! pip3 install elasticsearch
# ! pip3 install sentence_transformers
# ! pip3 install pandas

In [1]:
from dotenv import load_dotenv
import base64
import os
import http.client
import json

load_dotenv()

watsonx_api_key = os.getenv("WATSONX_APIKEY", None)
ibm_cloud_url = os.getenv("IBM_CLOUD_URL", None)
project_id = os.getenv("PROJECT_ID", None)
ibm_cloud_iam_url = os.getenv("IAM_IBM_CLOUD_URL", None)
chat_url = os.getenv("IBM_WATSONX_AI_INFERENCE_URL", None)

elastic_url = os.getenv("ELASTIC_URL", None)
elastic_api_key = os.getenv("ELASTIC_APIKEY", None)
### Encode image as base 64


# Embedding using Snowflake


In [18]:
from sentence_transformers import SentenceTransformer

model_name = 'Snowflake/snowflake-arctic-embed-l-v2.0'
model = SentenceTransformer(model_name)

In [ ]:
def embedding(sentences):
    # sentences = data['sentence']
    embeddings = model.encode(sentences)
    out = {
        'predictions': [
            {
                'fields': ['sentence', 'embedding'],
                'values': [[sentence, embedding.tolist()] for sentence, embedding in zip(sentences, embeddings)]
            }
        ]
    }

    

    return out


In [58]:
docs

[{'text': 'Yellowstone National Park is one of the largest national parks in the United States. It ranges from the Wyoming to Montana and Idaho, and contains an area of 2,219,791 acress across three different states. Its most famous for hosting the geyser Old Faithful and is centered on the Yellowstone Caldera, the largest super volcano on the American continent. Yellowstone is host to hundreds of species of animal, many of which are endangered or threatened. Most notably, it contains free-ranging herds of bison and elk, alongside bears, cougars and wolves. The national park receives over 4.5 million visitors annually and is a UNESCO World Heritage Site.'},
 {'text': 'Yosemite National Park is a United States National Park, covering over 750,000 acres of land in California. A UNESCO World Heritage Site, the park is best known for its granite cliffs, waterfalls and giant sequoia trees. Yosemite hosts over four million visitors in most years, with a peak of five million visitors in 2016.

In [ ]:
def simple_emb(sentences):
    # sentences = data['sentence']
    embeddings = model.encode(sentences)

    out = [[sentence, embedding.tolist()] for sentence, embedding in zip(sentences, embeddings)]
            
        

    

    return out

SyntaxError: invalid syntax (203924461.py, line 5)

In [ ]:
sim = simple_emb(docs)
sim/

[[{'text': 'Yellowstone National Park is one of the largest national parks in the United States. It ranges from the Wyoming to Montana and Idaho, and contains an area of 2,219,791 acress across three different states. Its most famous for hosting the geyser Old Faithful and is centered on the Yellowstone Caldera, the largest super volcano on the American continent. Yellowstone is host to hundreds of species of animal, many of which are endangered or threatened. Most notably, it contains free-ranging herds of bison and elk, alongside bears, cougars and wolves. The national park receives over 4.5 million visitors annually and is a UNESCO World Heritage Site.'},
  [0.06355016678571701,
   0.027205096557736397,
   -0.038933660835027695,
   0.03173014894127846,
   -0.03588595241308212,
   -0.04867343604564667,
   -0.030585603788495064,
   -0.05747586861252785,
   -0.04092332348227501,
   -0.04115191102027893,
   0.032935988157987595,
   0.02848370373249054,
   -0.008177823387086391,
   -0.00

In [65]:

docs = [
    {
        "text": "Yellowstone National Park is one of the largest national parks in the United States. It ranges from the Wyoming to Montana and Idaho, and contains an area of 2,219,791 acress across three different states. Its most famous for hosting the geyser Old Faithful and is centered on the Yellowstone Caldera, the largest super volcano on the American continent. Yellowstone is host to hundreds of species of animal, many of which are endangered or threatened. Most notably, it contains free-ranging herds of bison and elk, alongside bears, cougars and wolves. The national park receives over 4.5 million visitors annually and is a UNESCO World Heritage Site."
    },
    {
        "text": "Yosemite National Park is a United States National Park, covering over 750,000 acres of land in California. A UNESCO World Heritage Site, the park is best known for its granite cliffs, waterfalls and giant sequoia trees. Yosemite hosts over four million visitors in most years, with a peak of five million visitors in 2016. The park is home to a diverse range of wildlife, including mule deer, black bears, and the endangered Sierra Nevada bighorn sheep. The park has 1,200 square miles of wilderness, and is a popular destination for rock climbers, with over 3,000 feet of vertical granite to climb. Its most famous and cliff is the El Capitan, a 3,000 feet monolith along its tallest face."
    },
    {
        "text": "Rocky Mountain National Park  is one of the most popular national parks in the United States. It receives over 4.5 million visitors annually, and is known for its mountainous terrain, including Longs Peak, which is the highest peak in the park. The park is home to a variety of wildlife, including elk, mule deer, moose, and bighorn sheep. The park is also home to a variety of ecosystems, including montane, subalpine, and alpine tundra. The park is a popular destination for hiking, camping, and wildlife viewing, and is a UNESCO World Heritage Site."
    }
]

In [21]:
emb = embedding(docs)
value = emb['predictions'][0]['values']
print(emb['predictions'][0]['values'])

[[{'text': 'Yellowstone National Park is one of the largest national parks in the United States. It ranges from the Wyoming to Montana and Idaho, and contains an area of 2,219,791 acress across three different states. Its most famous for hosting the geyser Old Faithful and is centered on the Yellowstone Caldera, the largest super volcano on the American continent. Yellowstone is host to hundreds of species of animal, many of which are endangered or threatened. Most notably, it contains free-ranging herds of bison and elk, alongside bears, cougars and wolves. The national park receives over 4.5 million visitors annually and is a UNESCO World Heritage Site.'}, [0.06355016678571701, 0.027205096557736397, -0.038933660835027695, 0.03173014894127846, -0.03588595241308212, -0.04867343604564667, -0.030585603788495064, -0.05747586861252785, -0.04092332348227501, -0.04115191102027893, 0.032935988157987595, 0.02848370373249054, -0.008177823387086391, -0.0035257472191005945, 0.02161237597465515, -

<img src="output_structure_embedding.png" width="600"/>

# get dataframe from csv file

In [4]:
import pandas as pd
df = pd.read_csv("fish-description-files/Marine_Fish_Species_Full_Description_test.csv")
# df= df['Summary Description']
# df['Summary Description'] = df['Summary Description'].str.replace('\n', '', regex=False)
# df['Summary Description'].iloc[5]


## Elstic db

In [30]:
docs_with_embed = emb['predictions'][0]['values']


In [99]:
from elasticsearch import Elasticsearch

client = Elasticsearch(
    elastic_url,
    api_key=elastic_api_key
)

index_name = "fish-test"

mappings = {
    "properties": {
        "fish_name": {
            "type": "text"
        },
        "general_description": {
            "type": "semantic_text"
        },
        "image_links": {
            "type": "text"
        },
        # "embedding": {
        #     "type": "dense_vector",
        #     "dims": 768  # Adjust the dimension based on your model's output
        # }
    }
}


mapping_response = client.indices.put_mapping(index=index_name, body=mappings)
print(mapping_response)

{'acknowledged': True}


In [89]:
# from elasticsearch import Elasticsearch, helpers

# # Timeout to allow machine learning model loading and semantic ingestion to complete
# ingestion_timeout=300

# bulk_response = helpers.bulk(
#     client.options(request_timeout=ingestion_timeout),
#     docs,
#     index=index_name
# )
# print(bulk_response)

In [92]:
df

,Fish Name,Summary Description,Image Links
0,Brownbanded bamboo shark,This small carpet sharkis commonly found insha...,https://object-storage.example.com/brownbanded...
1,Whale shark,The world's largest livingshark and fish speci...,https://object-storage.example.com/whale_shark...
2,Zebra shark,A distinctive carpetshark with fivelongitudina...,https://object-storage.example.com/zebra_shark...
3,Coral catshark,A small bottom-dwellingshark found in coral re...,https://object-storage.example.com/coral_catsh...
4,Blacktip reef shark,A medium-sized reef shark easily identified by...,https://object-storage.example.com/blacktip_re...
...,...,...,...
86,Moorish idol,An iconic reef fishwith distinctiveblack and w...,https://object-storage.example.com/moorish_ido...
87,Little dragonfish,"A small, dragon-like fish withwing-like pector...",https://object-storage.example.com/little_drag...
88,Indian mackerel,"A small, schooling fishimportant in commercial...",https://object-storage.example.com/indian_mack...
89,Trumpetfish,An elongated fish with a tube-like snout used ...,https://object-storage.example.com/trumpetfish...


In [101]:
from elasticsearch.helpers import bulk

# Prepare bulk actions
actions = [
    {
        "_index": index_name,
        "_source": {
            "fish_name": row["Fish Name"],
            "general_description": row["Summary Description"],
            "image_links": row["Image Links"]
            

        }
    }
    for _, row in df.iterrows()
]

# Upload to Elasticsearch
success, errors = bulk(client, actions)
print(f"Success: {success}, Errors: {errors}")

ConnectionTimeout: Connection timed out

<img src="attachment:image.png" width="400"/>

In [9]:
print(emb['predictions'][0]['values'][0][1])

[0.06355016678571701, 0.027205096557736397, -0.038933660835027695, 0.03173014894127846, -0.03588595241308212, -0.04867343604564667, -0.030585603788495064, -0.05747586861252785, -0.04092332348227501, -0.04115191102027893, 0.032935988157987595, 0.02848370373249054, -0.008177823387086391, -0.0035257472191005945, 0.02161237597465515, -0.03241472318768501, -0.011762487702071667, 0.048839468508958817, -0.012501521967351437, -0.02542521059513092, 0.014871028251945972, -0.12251808494329453, -0.042574442923069, 0.00636927830055356, -0.024340476840734482, 0.04186815768480301, -0.018973998725414276, 0.03287295252084732, -0.03582679107785225, -0.12289444357156754, 0.015947693958878517, -0.03056233748793602, -0.00021605052461382002, -0.01288718730211258, 0.022227976471185684, -0.042233314365148544, -0.0009860893478617072, -0.04334075376391411, -0.019622592255473137, 0.04868042469024658, 0.021616823971271515, 0.03690490126609802, -0.012972572818398476, 0.07423292100429535, -0.04793231189250946, -0.0

input
des = description from watson of the test pic

1. test_pic_emb = embbing(des)
2. similarseach = (test_pic_emb, estic db)

# Promp wxai

In [5]:
conn_ibm_cloud_iam = http.client.HTTPSConnection(ibm_cloud_iam_url)
payload = "grant_type=urn%3Aibm%3Aparams%3Aoauth%3Agrant-type%3Aapikey&apikey="+watsonx_api_key
headers = { 'Content-Type': "application/x-www-form-urlencoded" }
conn_ibm_cloud_iam.request("POST", "/identity/token", payload, headers)
res = conn_ibm_cloud_iam.getresponse()
data = res.read()
decoded_json=json.loads(data.decode("utf-8"))
access_token=decoded_json["access_token"]

In [6]:
system_content ="""You always answer the questions with markdown formatting using GitHub syntax. The markdown formatting you support: headings, bold, italic, links, tables, lists, code blocks, and blockquotes. You must omit that you answer the questions with markdown.\n\nAny HTML tags must be wrapped in block quotes, for example ```<html>```. You will be penalized for not rendering code in block quotes.\n\nWhen returning code blocks, specify language.\n\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. \nYour answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don'\''t know the answer to a question, please don'\''t share false information."""
user_message = """Please provide a detailed description of the fish in the image focusing on the following aspects:
1. Body shape and size
2. Coloration patterns and markings
3. Distinctive features (fins, scales, head shape, etc.)
4. Any unique identifying characteristics
Return your answer in JSON format as follows:
{
    'body': 'description of body shape and approximate size',
    'colors': 'detailed description of colors and patterns',
    'features': 'description of distinctive features',
    'unique_marks': 'any unique identifying characteristics'
}"""

In [7]:
# load picture

pic = open("fish-pictures/fish-1.png","rb").read()
pic_base64 = base64.b64encode(pic)
pic_string = pic_base64.decode("utf-8")

In [8]:
# import os

# folder_path = "fish-pictures"  # Replace with your folder path

# for filename in os.listdir(folder_path):
#     file_path = os.path.join(folder_path, filename)
    
#     if os.path.isfile(file_path):
#         with open(file_path, "rb") as f:
#             data = f.read()
#             print(f"Read {filename}, size: {len(data)} bytes")

In [9]:
import requests
# system_content = """You always answer the questions with markdown formatting using GitHub syntax. The markdown formatting you support: headings, bold, italic, links, tables, lists, code blocks, and blockquotes. You must omit that you answer the questions with markdown.\n\nAny HTML tags must be wrapped in block quotes, for example ```<html>```. You will be penalized for not rendering code in block quotes.\n\nWhen returning code blocks, specify language.\n\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. \nYour answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don'\''t know the answer to a question, please don'\''t share false information."""
# user_message = """Please 1. Classify what object is this 2. Give a scale of 1-10 how damaged is the car 3. Do we need to change the parts or can fix. Please provide some descriptions\nAnswer in JSON with format {{'object': 'object_name', 'damage_scale': num, 'description': 'some_description'}}"""
body = {
   "messages": [
      {
         "role": "system",
         "content": system_content
      },
      {
         "role": "user",
         "content": [
            {
               "type": "text",
               "text": user_message,
            },
            {
               "type": "image_url",
               "image_url": {
                  "url": f"data:image/jpeg;base64, {pic_string}"
               }
            }
         ]
      }
   ],
   "project_id": project_id,
   # "model_id": "meta-llama/llama3-llava-next-8b-hf",
   # "model_id": "meta-llama/llama-3-2-11b-vision-instruct",
   "model_id": "meta-llama/llama-3-2-90b-vision-instruct",
   "decoding_method": "greedy",
   "repetition_penalty": 1.1,
   "max_tokens": 900
}

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": f"Bearer {access_token}"
}

response = requests.post(
    chat_url,
    headers=headers,
    json=body
)

if response.status_code != 200:
    raise Exception("Non-200 response: " + str(response.text))

data = response.json()

print('---- RESPONSE ----')
print(data['choices'][0]['message']['content'])

/Users/tanyatonoran/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


---- RESPONSE ----
\{
    'body': 'The fish has a compact, oval-shaped body with a size of approximately 3-5 inches (7-13 cm) in length.',
    'colors': 'The fish features a vibrant orange body with three white stripes, one behind the eye, one in the middle of the body, and one at the base of the tail. The fins are orange with black edges, and the tail is rounded.',
    'features': 'The fish has a small mouth and a pointed snout. It has large, rounded fins that are used for maneuverability and balance in the water.',
    'unique_marks': 'The fish has a distinctive anemone-like pattern on its body, which is actually a result of its symbiotic relationship with the sea anemone. The anemone provides the fish with protection from predators, while the fish helps to keep the anemone clean and free of parasites.'
\}


In [85]:
input_text_description = data['choices'][0]['message']['content']

# query from elastic

In [ ]:
from elasticsearch import Elasticsearch


retriever_object = {
    "standard": {
        "query": {
            "semantic": {
                "field": "general_description",
                "query": input_text_description
            }
        }
    }
}

search_response = client.search(
    index="fish-test",
    retriever=retriever_object,
)
print(search_response['hits']['hits'])

[{'_index': 'fish-test', '_id': 'EUnh-JcBcKYh01VyX7yD', '_score': 27.791382, '_source': {'fish_name': 'Tomato clownfish', 'general_description': 'A bright red-orangeclownfish with asingle white barbehind the head.One of the largerclownfish speciescommonly found inanemones.', 'image_links': 'https://object-storage.example.com/tomato_clownfish_1.jpg, https://object-storage.example.com/tomato_clownfish_2.jpg'}}, {'_index': 'fish-test', '_id': 'tEng-JcBcKYh01Vy5bsj', '_score': 27.791382, '_source': {'fish_name': 'Tomato clownfish', 'general_description': 'A bright red-orangeclownfish with asingle white barbehind the head.One of the largerclownfish speciescommonly found inanemones.', 'image_links': 'https://object-storage.example.com/tomato_clownfish_1.jpg, https://object-storage.example.com/tomato_clownfish_2.jpg'}}, {'_index': 'fish-test', '_id': 'skng-JcBcKYh01Vy5bsi', '_score': 26.930103, '_source': {'fish_name': 'Powderblue surgeonfish', 'general_description': 'A vibrantsurgeonfish wit

In [ ]:
# 

# p mew code: for ingestion

In [ ]:
es = client

In [ ]:
def creating_index_body(index_name, dimension=768):
    # Create index with mapping for embeddings
    # index_name = "test-index-2"
    create_index_body = {
        "settings": {
            "index": {
            "analysis": {
                "analyzer": {
                "analyzer_shingle": {
                    "tokenizer": "icu_tokenizer",
                    "filter": [
                    "filter_shingle"
                    ]
                }
                },
                "filter": {
                "filter_shingle": {
                    "type": "shingle",
                    "max_shingle_size": 3,
                    "min_shingle_size": 2,
                    "output_unigrams": "true"
                }
                }
            }
            }
        },  
        "mappings": {
            "properties": {
                "en_character": {"type": "text"},
                "en_character_description": {"type": "text", "analyzer": "analyzer_shingle"},
                "vector_en": {
                    "type": "dense_vector",
                    "dims": dimension,
                    "index": True,
                    "similarity": "cosine"
                }
            }
        }
    }
    if not es.indices.exists(index=index_name):
        print(f"Index '{index_name}' does not exist. Creating a new one")
    else:
        response = es.indices.delete(index=index_name)
        print(f"Index '{index_name}' deleted successfully.")
    es.indices.create(index=index_name, body=create_index_body)

In [ ]:
def ingestion_data_to_elastic(index_name, dataframe):
    # Index the documents with semantic embeddings and raw text
    for i in range(len(dataframe)):
        print(dataframe.iloc[i])
        # Generate embeddings
        Material = dataframe.iloc[i]["Material"]
        description_th = dataframe.iloc[i]["Description TH"]
        description_en = dataframe.iloc[i]["Description EN"]
        enhanced_description = dataframe.iloc[i]["enhanced_description"]
        enhanced_description_model = dataframe.iloc[i]["enhanced_description_model"]
        price = dataframe.iloc[i]["Price (Baht)"]
        location_to_stock = dataframe.iloc[i]["Location : Stock"]
        # raw_promo_description = dataframe.iloc[i]["Bonus Buy Description"]
        enhanced_promo_description = dataframe.iloc[i]["enhanced_promo_description"]
        brand_name = dataframe.iloc[i]["Brand Name"]
        text_to_encode = f"""
        Description (EN): {description_en}
        Description (TH): {description_th}
        Brand Name: {brand_name}
        Enhanced Description: {enhanced_description}
        """
        embedding = embedding(text_to_encode)

        # Index document with both raw text and embeddings
        res = es.index(index=index_name, id=i, body={
            'Material': Material,
            'description_th': description_th,
            'description_en': description_en,
            'enhanced_description': enhanced_description,
            'enhanced_description_model': enhanced_description_model,
            # 'raw_promo_description': raw_promo_description,
            'enhanced_promo_description': enhanced_promo_description,
            'location_to_stock': location_to_stock,
            'brand_name': brand_name,
            'price': price,
            'vector_en': embedding  # Store embedding
        })
        print(f"Document {i} indexing result: {res['result']}")

# title,full_description,actual_price,discount
# Material,Description TH,Description EN,Brand,Brand Name,Mat. Group,Material Type,enhanced_description,Price (Baht),Location : Stock,Bonus Buy Description,enhanced_promo_description
## Search


In [ ]:
index_name = "fish-test"
print("starting to index...")
creating_index_body(index_name, 1024)

ingestion_data_to_elastic(index_name, df)

starting to index...
Index 'fish-test' deleted successfully.
This small carpet sharkis commonly found inshallow coastal watersand coral reefsthroughout the IndoWest Pacific. Youngindividuals have distinctbrown banding thatfades to solid brown asthey mature. They arenocturnal feeders thatcan survive up to 12hours out of water.


TypeError: string indices must be integers